In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
from pprint import pprint
from concurrent.futures import ProcessPoolExecutor
from itertools import chain
import nltk
import re
import os
from pathlib import Path

## 1. Set up file path and Read Files

In [3]:
GLOBAL_PATH="../_data/CORD-19-research-challenge"

In [4]:
cord_19_embeddings_df=pd.read_csv(GLOBAL_PATH+'/cord_19_embeddings_4_17/cord_19_embeddings_4_17.csv')

In [5]:
cord_19_embeddings_df.head()

,xqhn0vbp,0.2771094739437103,-2.9749464988708496,1.10358726978302,9.312440872192383,1.3013277053833008,-4.281131267547607,-0.5318851470947266,-4.497644424438477,-0.6967141628265381,...,-0.31924888491630554,4.262275695800781,2.845273971557617,-1.656341552734375,-2.5844340324401855,-4.678825855255127,3.175534248352051,4.8904242515563965,-1.488284707069397,-1.3013025522232056
0,gi6uaa83,-0.344760,-4.762074,3.647769,2.616938,3.856543,-0.460272,0.899565,2.284138,0.590927,...,-0.185362,0.603534,1.475987,1.804034,2.819780,-4.039684,-3.027682,0.997251,-0.661524,1.590745
1,le0ogx1s,-3.257521,-1.720686,1.438893,-1.614458,-3.205178,1.125262,-1.420612,-4.270585,-2.557855,...,-1.691883,0.594955,1.336509,0.036567,-2.065922,0.360089,0.832659,2.954574,-4.601091,-1.097578
2,fy4w7xz8,0.920366,-3.546179,-2.537739,6.372102,0.025263,-1.991629,-0.612892,0.983194,-2.316126,...,-0.412801,0.831824,2.101388,3.283052,-3.075248,-4.381618,1.925288,4.607265,-1.254582,-1.320425
3,0qaoam29,-1.145982,-5.231421,-1.958305,5.035599,-1.601498,-0.756577,1.757929,1.490937,0.192156,...,0.131644,3.836460,1.115987,1.524282,-2.017589,-5.102140,3.864315,1.679577,1.871223,-0.295825
4,qj4dh6rg,-1.728899,-5.419925,-0.482808,9.170734,-1.273682,-1.007223,1.836062,3.512957,1.623566,...,-0.164270,4.492704,1.499324,4.371634,-2.533002,-4.532452,2.206654,1.812408,0.771027,-1.834923


In [4]:
metadata_path = f'{GLOBAL_PATH}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})

In [1]:
meta_df.shape

NameError: name 'meta_df' is not defined

## 2. Read BioXiv paper for testing training generator functions 

In [42]:
#biorxiv = '/biorxiv_medrxiv' #2278 in bioxiv
comm_used = '/comm_use_subset/comm_use_subset'
load_json = glob.glob(f'{GLOBAL_PATH}/**/*.json', recursive=True)


In [43]:
len(load_json)

68204

In [8]:
print("There are: ",len(load_json)," pdf Json in Path")
print("===================================================")
all_files = []
for filename in load_json:
    #filename = biorxiv_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files.append(file)

file = all_files[0]
print("Dictionary keys:", file.keys())

There are:  2278  pdf Json in BioXiv
Dictionary keys: dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])


In [35]:
pprint(file['abstract'])

[{'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Aims: To determine analytical capabilities of a commonly used '
          'faecal immunochemical test (FIT) to detect haemoglobin (Hb) in the '
          'context of NICE guidance DG30, and the likely use of FIT to '
          'reprioritise patients delayed by the COVID-19 pandemic.'},
 {'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Methods: Data obtained from independent verification studies and '
          'clinical testing of the HM-JACKarc FIT method in routine primary '
          'care practice were analysed to derive analytical performance '
          'characteristics.'},
 {'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Results: Detection capabilities for the FIT method were 0.5 µg/g '
          '(limit of blank), 1.1 (limit of detection) and 15.0 µg/g (limit of '
          'quantification). 31 of 33 (94%) non-homogenised specimens analysed '
          '

In [34]:
print("body_text content:")
pprint(file['body_text'][:2], depth=3)

body_text content:
[{'cite_spans': [{...}, {...}, {...}],
  'ref_spans': [],
  'section': 'Introduction',
  'text': 'Colorectal cancer is globally the third most incident malignancy '
          '(1) . It is surgically treatable with improved long-term outcomes '
          'if diagnosis is at an early stage (2) . Most developed countries, '
          'including the UK, operate colorectal screening programmes using '
          'faecal occult blood testing. Screen detected cancers benefit from '
          'early diagnosis and treatment, with associated improved survival '
          '(2) . Faecal immunochemical tests (FIT) have largely replaced the '
          'traditional guaiac based faecal occult blood tests due to the '
          'increased specificity of FIT.'},
 {'cite_spans': [{...}, {...}],
  'ref_spans': [],
  'section': 'Introduction',
  'text': 'To complement the UK Bowel Cancer Screening Programme (UKBCSP), the '
          '2017 DG30 NICE guidance (3) recommended the use of FIT

In [40]:
texts = [(di['section'], di['text']) for di in file['body_text']]
texts_di = {di['section']: "" for di in file['body_text']}
for section, text in texts:
    texts_di[section] += text

pprint(list(texts_di.keys()))

['Introduction',
 'Setting and analytical method',
 'Method detection capability estimates and immunoassay reproducibility',
 'Effects of sample homogenisation',
 'Stability of specimens within the collection device',
 'Within patient serial sampling',
 'Statistical analysis',
 'Detection capabilities and immunoassay reproducibility',
 'Discussion',
 'Implications for research and practice.',
 'Concluding remarks.']


In [43]:
body = ""

for section, text in texts_di.items():
    body += section
    body += "\n\n"
    body += text
    body += "\n\n"

print(body[:3000])

Introduction

Colorectal cancer is globally the third most incident malignancy (1) . It is surgically treatable with improved long-term outcomes if diagnosis is at an early stage (2) . Most developed countries, including the UK, operate colorectal screening programmes using faecal occult blood testing. Screen detected cancers benefit from early diagnosis and treatment, with associated improved survival (2) . Faecal immunochemical tests (FIT) have largely replaced the traditional guaiac based faecal occult blood tests due to the increased specificity of FIT.To complement the UK Bowel Cancer Screening Programme (UKBCSP), the 2017 DG30 NICE guidance (3) recommended the use of FIT for faecal haemoglobin (Hb) detection in patients presenting to primary care with low risk abdominal symptoms. The adoption of FIT in primary care has been slow, with notable variation in uptake and implementation across the UK (4). The Oxford University Hospitals Trust (OUH) adopted FIT prior to the DG30 guidanc

## 3. Flatten the Json file to create the training set 

In [9]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'

first_row = FileReader(load_json[0])
print(first_row)

f6b29be971089bfe0916c64ab9fbddcec38a7436: Aims: To determine analytical capabilities of a commonly used faecal immunochemical test (FIT) to detect haemoglobin (Hb) in the context of NICE guidance DG30, and the likely use of FIT to reprioritis... Colorectal cancer is globally the third most incident malignancy (1) . It is surgically treatable with improved long-term outcomes if diagnosis is at an early stage (2) . Most developed countries, inc...


In [10]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'title': []}
for idx, entry in enumerate(load_json):
    if idx % (len(load_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(load_json)}')
    
    try:
        content = FileReader(entry)
    except Exception as e:
        continue  # invalid paper format, skip
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['abstract'].append(content.abstract)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    
    # add the title information, add breaks when needed
    dict_['title'].append(meta_data['title'])

    
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'title'])
df_covid.head()

Processing index: 0 of 2278
Processing index: 227 of 2278
Processing index: 454 of 2278
Processing index: 681 of 2278
Processing index: 908 of 2278
Processing index: 1135 of 2278
Processing index: 1362 of 2278
Processing index: 1589 of 2278
Processing index: 1816 of 2278
Processing index: 2043 of 2278
Processing index: 2270 of 2278


,paper_id,abstract,body_text,title
0,f6b29be971089bfe0916c64ab9fbddcec38a7436,Aims: To determine analytical capabilities of ...,Colorectal cancer is globally the third most i...,37232 Faecal immunochemical testing (FIT): ...
1,005d189d5bd7ac01aee65e934fd3d5186a3f7b27,The rapid outbreak of the new Coronavirus pand...,The outbreak of infectious diseases has always...,36952 Relationship between Average Daily Te...
2,f905f78b32f63c6d14a79984dfb33f1b358b8ab4,New anti-AIDS treatments must be continually d...,"In the absence of a curative treatment, the hi...",37478 Multimerization of HIV-1 integrase hi...
3,607e0074d8ff40c272b958c2fe48793fedfc785e,,"the author/funder, who has granted medRxiv a l...",36290 Virus shedding patterns in nasopharyn...
4,72e25b728c6c62fb3a4e2c59c8ee48de4b5ee452,"Recently classified as a pandemic by WHO, nove...",The pandemic of COVID-19 is taking a troll in ...,37135 Phylogenetic Analysis of the Novel Co...


In [11]:
len(df_covid["body_text"])

2278

In [14]:
#df_covid["body_text"][0]

## 4. Cleaning the Text 

In [55]:
import nltk
from nltk.tokenize import sent_tokenize
import sentencepiece as spm

In [52]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

In [15]:
def normalize_text(text):
    # lowercase text
    text = str(text).lower()
    # remove non-UTF
    text = text.encode("utf-8", "ignore").decode()
    # remove punktuation symbols
    #text = " ".join(regex_tokenizer.tokenize(text))
    return text

def sentencise_text(text):
    text = sent_tokenize(text)
    return text 

def count_lines(filename):
    count = 0
    with open(filename) as fi:
        for line in fi:
            count += 1
    return count

In [16]:
normalize_text(df_covid["body_text"][0])

'colorectal cancer is globally the third most incident malignancy (1) . it is surgically treatable with improved long-term outcomes if diagnosis is at an early stage (2) . most developed countries, including the uk, operate colorectal screening programmes using faecal occult blood testing. screen detected cancers benefit from early diagnosis and treatment, with associated improved survival (2) . faecal immunochemical tests (fit) have largely replaced the traditional guaiac based faecal occult blood tests due to the increased specificity of fit.\nto complement the uk bowel cancer screening programme (ukbcsp), the 2017 dg30 nice guidance (3) recommended the use of fit for faecal haemoglobin (hb) detection in patients presenting to primary care with low risk abdominal symptoms. the adoption of fit in primary care has been slow, with notable variation in uptake and implementation across the uk (4). the oxford university hospitals trust (ouh) adopted fit prior to the dg30 guidance to comply

In [18]:
PRC_DATA_FPATH = "pretrain/data/proc_dataset.txt" #@param {type: "string"}
with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in df_covid["body_text"]:
        fo.write(normalize_text(l)+"\n")

### saving sentence per line 

In [19]:
nltk.download('punkt')

extra_abbreviations = ['ps',  'inc', 'Corp', 'Ltd', 'Co', 'pkt', 'Dz.Ap', 'Jr', 'jr', 'sp', 'Sp', 'poj',  'pseud', 'krypt', 'sygn', 'Dz.U', 'ws', 'itd', 'np', 'sanskryt', 'nr', 'gł', 'Takht', 'tzw', 't.zw', 'ewan', 'tyt', 'oryg', 't.j', 'vs', 'l.mn', 'l.poj' ]

position_abbrev = ['Ks', 'Abp', 'abp','bp','dr', 'kard', 'mgr', 'prof', 'zwycz', 'hab', 'arch', 'arch.kraj', 'B.Sc', 'Ph.D', 'lek', 'med', 'n.med', 'bł', 'św', 'hr', 'dziek' ]

quantity_abbrev = [ 'mln', 'obr./min','km/godz', 'godz', 'egz', 'ha', 'j.m', 'cal', 'obj', 'alk', 'wag' ] # not added: tys.

actions_abbrev = ['tłum','tlum','zob','wym', 'pot', 'ww', 'ogł', 'wyd', 'min', 'm.i', 'm.in', 'in', 'im','muz','tj', 'dot', 'wsp', 'właść', 'właśc', 'przedr', 'czyt', 'proj', 'dosł', 'hist', 'daw', 'zwł', 'zaw' ]

place_abbrev = ['Śl', 'płd', 'geogr']

lang_abbrev = ['jęz','fr','franc', 'ukr', 'ang', 'gr', 'hebr', 'czes', 'pol', 'niem', 'arab', 'egip', 'hiszp', 'jap', 'chin', 'kor', 'tyb', 'wiet', 'sum']

military_abbrev = ['kpt', 'kpr', 'obs', 'pil', 'mjr','płk', 'dypl', 'pp', 'gw', 'dyw', 'bryg', 'ppłk', 'mar', 'marsz', 'rez', 'ppor', 'DPanc', 'BPanc', 'DKaw', 'p.uł']

extra_abbreviations= extra_abbreviations + position_abbrev + quantity_abbrev + place_abbrev + actions_abbrev + place_abbrev + lang_abbrev+military_abbrev

sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')
sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)

sent_tokenize = sentence_tokenizer.tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/catherinewang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
def flatten(iterable):
    return chain.from_iterable(iterable)
    

In [36]:
def process_line(line):
    try:
        #doc = json.loads(line)
        txt = re.sub("\s+", " ", line)
        sentences = [s for s in sent_tokenize(txt)]
        windowed_sentences = []
        for snt in range(len(sentences)):
            windowed_sentences.append(" ".join(sentences[snt: snt + 4]))
        return windowed_sentences
    except:
        # print(f"Could not parse line \n{line}\n")
        return []

In [37]:
def process_file(file_path):
    print(f"Processing {file_path}")
    lines = Path(file_path).read_text("utf-8").split("\n")
    with ProcessPoolExecutor(10) as pool:
        return list(flatten(pool.map(process_line, lines)))

In [38]:
files = [f for f in glob.glob("pretrain/data/**.txt", recursive=True) if os.path.isfile(f)]

In [39]:
files

['pretrain/data/proc_dataset.txt']

In [40]:
buffer, BUFFER_SIZE = [], 100000
with open("biorxiv_medrxiv.train.sliding4-v2.txt", "wt") as file:
    for i, sentence in enumerate(flatten(process_file(f) for f in files)):
        if len(buffer) >= BUFFER_SIZE:
            file.write("\n".join(buffer))
            buffer.clear()
            print(i, end="\r")
        buffer.append(sentence)
    if len(buffer) > 0:
        file.write("\n".join(buffer))
        buffer.clear()

Processing pretrain/data/proc_dataset.txt


## 4. Run all dataset 